In [22]:
import nltk
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer




In [9]:
df = pd.read_csv('output.csv')

In [10]:
df.head()

,comment
0,"Always good food from Hoagies, love their chic..."
1,Fried pickles and honey mayo - amazing
2,Great value and yummy food. Falafal burger was...
3,Yummy
4,The food is consistently excellent and the sta...


In [12]:
df.shape

(72736, 1)

In [14]:
# detect rows with missing reviews
df.isnull().sum()

comment    0
dtype: int64

In [17]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove whitespaces
    text = text.strip()
    return text
    
    

In [19]:
def tokenize(text):
    # split the text into individual words
    return text.split()

In [23]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    #Remove common words that usually don’t carry important meaning.
    return [word for word in tokens if not word in stop_words]

In [25]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [26]:
def rejoin_tokens(tokens):
    return ' '.join(tokens)

In [27]:
def preprocess_pipeline(text):
    # Chain your preprocessing functions
    text = clean_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize_tokens(tokens)
    text = rejoin_tokens(tokens)
    return text



In [28]:
# Apply the preprocessing pipeline to your text column
df['cleaned_review'] = df['comment'].apply(preprocess_pipeline)

In [29]:
df

,comment,cleaned_review
0,"Always good food from Hoagies, love their chic...",always good food hoagy love chicken burger fry...
1,Fried pickles and honey mayo - amazing,fried pickle honey mayo amazing
2,Great value and yummy food. Falafal burger was...,great value yummy food falafal burger yum reco...
3,Yummy,yummy
4,The food is consistently excellent and the sta...,food consistently excellent staff always reall...
...,...,...
72731,Really enjoyed our food. All freshly made and ...,really enjoyed food freshly made tasty ready q...
72732,This place is rotten 12inch munchie box in bin...,place rotten inch munchie box bin plus menu bo...
72733,What a gem of a takeaway. Food always hot and ...,gem takeaway food always hot delicious staff a...
72734,Fantastic !South indian garlic chilli is th...,fantastic south indian garlic chilli best bett...
